In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

In [3]:
#define tensors in pytorch - same as arrays in numpy - with tensor
X = torch.tensor(([2,9], [1,5], [3,6]), dtype=torch.float) #3x2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) #3 by 1 tensor
x_test = torch.tensor(([4, 8]))

#can check the size with .size()
X.size()

torch.Size([3, 2])

In [4]:
#apply scaling
#max function returns max element and indice
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(x_test, 0)

#divides all the elements by X_max - in this case X_max return largest single value in the matrices i.e. 9
X = torch.div(X, X_max)
x_test = torch.div(x_test, xPredicted_max)
y = y/100
print(X)

tensor([[0.6667, 1.0000],
        [0.3333, 0.5556],
        [1.0000, 0.6667]])


In [15]:
#Building The Neural Network
#nn.module is the base class for any module in PyTorch
class NeuralNetwork(nn.Module):
    def __init__(self,):
        super(NeuralNetwork, self).__init__()
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenNodes = 3
        
        self.W1 = torch.randn(self.inputSize, self.hiddenNodes) #Tensor is 2 by 3 since (1 by 2) * (2 by 3) gives desired (1 by 3)
        self.W2 = torch.randn(self.hiddenNodes, self.outputSize) #Tesnsor is 3 by 1
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1)
        self.z2 = self.sigmoid(self.z)
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3)
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1-s)
    
    def backward(self, x, y, o):
        self.o_error = y - o
        self.o_delta = self.o_error * self.sigmoidPrime(o)
        #torch.t is the transpose operation
        #recall in the second wave of operations we do sigmoid(z2 * W2)
        #thus this can be represented as two nodes in a computation graph, the sigmoid and the matrix multiplication
        #partial derivative at the matrix multiplication level
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        #partial derivative at the sigmoid level
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
    
         
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(x_test))
        print ("Output: \n" + str(self.forward(x_test)))
    

In [16]:
#Training
NN = NeuralNetwork()
for i in range(100):
    #NN(X) automatically calls the forward so there is no need to explicity call forward
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

#0 Loss: 0.005908905062824488
#1 Loss: 0.005748603492975235
#2 Loss: 0.005600081756711006
#3 Loss: 0.005462279077619314
#4 Loss: 0.005334220826625824
#5 Loss: 0.005215070676058531
#6 Loss: 0.005104073788970709
#7 Loss: 0.005000508856028318
#8 Loss: 0.004903781693428755
#9 Loss: 0.00481331767514348
#10 Loss: 0.004728619009256363
#11 Loss: 0.004649232141673565
#12 Loss: 0.004574731923639774
#13 Loss: 0.004504744429141283
#14 Loss: 0.004438930191099644
#15 Loss: 0.004376981407403946
#16 Loss: 0.0043186102993786335
#17 Loss: 0.004263560753315687
#18 Loss: 0.004211593884974718
#19 Loss: 0.0041624936275184155
#20 Loss: 0.0041160574182868
#21 Loss: 0.004072108771651983
#22 Loss: 0.0040304833091795444
#23 Loss: 0.0039910185150802135
#24 Loss: 0.003953570034354925
#25 Loss: 0.0039180186577141285
#26 Loss: 0.0038842360954731703
#27 Loss: 0.003852112917229533
#28 Loss: 0.0038215413223952055
#29 Loss: 0.0037924284115433693
#30 Loss: 0.0037646889686584473
#31 Loss: 0.003738240571692586
#32 Loss: 0.

/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type NeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


RuntimeError: Expected object of scalar type Long but got scalar type Float for argument #2 'mat2' in call to _th_mm